In [13]:
import codecs
import os
from collections import OrderedDict,defaultdict,Counter
import time
import numpy as np
from pandas import Series, DataFrame 

input_path = 'C:\\Users\\user\\Desktop\\RF result\\顯著特徵尋找\\特徵值\\'
vector_path = 'C:\\Users\\user\\Desktop\\RF result\\vector\\'
SC_feature_path = "D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\平衡語料庫\\" #平衡語料庫語言特徵
FC_feature_path = "D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\自由中國\\" #自由中國語言特徵
lei_feature_path = "D:\\課業相關\\論文資料\\論文程式\\language_feature\\最終版\\雷震文本\\" #雷震文本語言特徵

classification_name = ['雷震','殷海光','夏道平','傅正','龍平甫','蔣勻田','朱伴耘','胡適','羅鴻詔']
classification_topic = ['社論','文章','日記']
feature_condicate = ['高頻','bigram','trigram','標點','N+N','N+V','VH+N','D+V','否定','程度','情態']


#建立作者索引，提取各作者文章內容及索引
author_index = []
for index,name in enumerate(classification_name): #建立作者索引
    author_index.append((name,index))
author_index = OrderedDict(author_index) #作者索引排序(依文本數量高到低)

topic_index = []
for index,name in enumerate(classification_topic): #建立作者索引
    topic_index.append((name,index))
topic_index = OrderedDict(topic_index) #作者索引排序(依文本數量高到低)

In [14]:
#輸入介面
def find_input(find):
    
    feature_file_path = '' #符合尋找的特徵檔案路徑
    feature_file_name = '' #符合尋找的特徵檔案名稱
    condicate_label = '' #候選類別名稱
    condicate_index = '' #候選類別索引

    temp = find.split()
    
    if len(temp) != 3:
        print ('請輸入正確值')
        return False
    
    if temp[0] == 'SC':
        feature_file_path = SC_feature_path
    elif temp[0] == 'FC':
        feature_file_path = FC_feature_path
    elif temp[0] == 'lei':
        feature_file_path = lei_feature_path
    else:
        print ('母體選項不符合')
        return False
    
    feature_file_name = [file for file in os.listdir(feature_file_path) if temp[1] in file]
    if len(feature_file_name) == 0:
        print ('輸入的語言特徵不在範圍內')
        return False
    feature_file_name = feature_file_name[0]

    if temp[2] == 'name':
        condicate_label = classification_name
        condicate_index = author_index
    elif temp[2] == 'topic':
        condicate_label = classification_topic
        condicate_index = topic_index
    else:
        print ('領域選項不符合')
        return False
        
    return feature_file_path+feature_file_name,condicate_label,condicate_index

In [80]:
#找出各類別重點詞組
#觀察獨特性、鄰近性、群體性及特殊關係(gini特別高)
for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        
        #if file.split('.')[0].split()[1] not in feature_condicate:
        #    continue
            
        print (file.split('.')[0])
        print ('詞組','類別','分數','平均','標準差','數值0次數','綜合','類別數據')
        
        feature_path,condicate_label,condicate_index = find_input(file.split('.')[0])
        
        feature_dict = {} #特徵
        feature = []
        with codecs.open(feature_path,'rb','utf8') as ff: #抓取基準特徵
            count = 0
            for i in ff.readlines():
                if '\ufeff' in i: #去掉開頭BOM
                    i = i.replace('\ufeff','')
                if i.strip() != '':
                    feature.append(i.strip().split(',')[0])
                    feature_dict[i.strip().split(',')[0]] = count
                    count += 1
        
        class_vector = defaultdict(list) #各類別文本向量
        with codecs.open(vector_path+file.split('.')[0]+'.csv','rb','utf8') as ff:
            for i in ff.readlines():
                if '\ufeff' in i: #去掉開頭BOM
                    i = i.replace('\ufeff','')
                if i.strip() != '':
                    i = i.strip().split(',')
                    class_vector[condicate_label[int(i[0])]].append([float(j) for j in i[1:]])
        
        content = f.readlines()
        
        '''all_feature = []
        
        feature_importance = defaultdict(list)
        name = ''
        count = 2
        for line in content:
            if line[0] == '#':
                name = line[1:].strip()
            #elif name != '' and '雷震' not in name:
            #    continue
            elif count > 0:
                count = count - 1
            elif line.strip() == '':
                count = 2
            else:
                temp_line = line.strip().split()
                if len(temp_line) != 2:
                    continue
                if float(temp_line[1]) > 0.01:
                    feature_importance[name].append((temp_line[0],float(temp_line[1])))
                    if temp_line[0] not in all_feature:
                        all_feature.append(temp_line[0])

        feature_num = defaultdict(int) #以排序作為詞組分數
        for i in all_feature:
            for k,v in feature_importance.items():
                check = True
                for index,element in enumerate(v):
                    if element[0] == i:
                        feature_num[i] += (index + 1)
                        check = False
                if check:
                    feature_num[i] += 10000 #懲罰數值
                        
        so = sorted(feature_num.items(), key=lambda d:d[1], reverse = False)
        for i,e in enumerate(so):
            if i >= 10:
                break
            #print (e[0],e[1])

        print ()'''
        
        feature_importance = defaultdict(list)
        name = ''
        count = 2
        for line in content:
            if line[0] == '#':
                name = line[1:].strip()
            #elif name != '' and '雷震' not in name:
            #    continue
            elif count > 0:
                count = count - 1
            elif line.strip() == '':
                count = 2
            else:
                temp_line = line.strip().split()
                if len(temp_line) == 2:# and float(temp_line[1]) > 0.01:
                    feature_importance[name].append((temp_line[0],float(temp_line[1])))
                    
        #詞組 : [兩兩類別,排序,切割分數,
        # (A類別文本該詞組數值最大、最小、平均、標準差、數值0次數、綜合),(B類別文本該詞組數值最大、最小、平均、標準差、數值0次數、綜合)]
        feature_appear = defaultdict(list) 
        for k,v in feature_importance.items():
            for index,e in enumerate(v):
                a_class = np.array(class_vector[k.split()[0]]) 
                b_class = np.array(class_vector[k.split()[1]])
                
                def class_caculator(c): 
                    class_max = max(c[:,feature_dict[e[0]]])
                    class_min = min(c[:,feature_dict[e[0]]])
                    class_mean = np.mean(c, axis=0)[feature_dict[e[0]]]
                    class_std = np.std(c, axis=0)[feature_dict[e[0]]]
                    class_zero = Counter(c[:,feature_dict[e[0]]].tolist())[0]/c.shape[0]
                    class_sum = sum(c[:,feature_dict[e[0]]])
                    
                    return (class_max,class_min,class_mean,class_std,class_zero,class_sum)
                    
                feature_appear[e[0]].append([k,index,e[1],class_caculator(a_class),class_caculator(b_class)])
                
        for i in feature:
            #if len(feature_appear[i]) == (len(condicate_label)*(len(condicate_label)-1))/2: 
            #    print (i)
            #print (len(feature_appear[i]),condicate_label)
            #print (i,feature_appear[i])
            #print (i,len(feature_appear[i]),sum([j[1] for j in feature_appear[i]]))
            for j in condicate_label:
                temp_check = []
                for k in feature_appear[i]:
                    if j in k[0]:
                        temp_check.append(k)
                if len(temp_check) == len(condicate_label)-1 and sum([k[1] for k in temp_check]) < 50:
                    compare = len(condicate_label)-1
                    def compare_value(com,num):
                        compare = com
                        for l in temp_check:
                            if l[0].split()[0] == j:
                                if l[3][num] > l[4][num]:
                                    compare -= 1
                            elif l[0].split()[1] == j:
                                if l[3][num] < l[4][num]:
                                    compare -= 1
                        return compare
                    
                    class_value = ''
                    if temp_check[0][0].split()[0] == j:
                        class_value = temp_check[0][3]
                    elif temp_check[0][0].split()[1] == j:
                        class_value = temp_check[0][4]
                    
                    print (i,j,sum([k[1] for k in temp_check]),compare_value(compare,2),compare_value(compare,3)
                           ,compare_value(compare,4),compare_value(compare,5),class_value)

        print ()
        #time.sleep(3)
print ('END')

FC bigram name
詞組 類別 分數 平均 標準差 數值0次數 綜合 類別數據
這種 殷海光 22 0 1 8 0 (6258.0, 638.0, 2831.5185185185187, 1526.0526026167515, 0.0, 76451.0)
這一 夏道平 45 1 0 8 0 (9202.0, 0.0, 3207.9736842105262, 1970.6025592888841, 0.02631578947368421, 121903.0)
這一 龍平甫 33 8 8 0 8 (686.0, 0.0, 66.975609756097555, 133.89848499204459, 0.7073170731707317, 2746.0)
的一 胡適 35 0 0 7 0 (5284.0, 0.0, 1910.5999999999999, 1360.6393497176246, 0.06666666666666667, 57318.0)
他的 胡適 49 0 0 7 0 (5442.0, 0.0, 2001.6666666666667, 1670.4153841352027, 0.13333333333333333, 60050.0)
都是 羅鴻詔 49 0 2 8 0 (3365.0, 0.0, 1359.7407407407406, 754.83476693726084, 0.07407407407407407, 36713.0)
的事 殷海光 32 0 1 8 0 (1901.0, 0.0, 950.62962962962968, 559.26387783229029, 0.1111111111111111, 25667.0)
事實上 傅正 16 0 0 8 0 (2385.0, 0.0, 884.10000000000002, 584.50422581877024, 0.05, 17682.0)
我的 胡適 34 0 1 8 0 (5556.0, 0.0, 1118.3666666666666, 1099.8494891979035, 0.2, 33551.0)

FC D+V name
詞組 類別 分數 平均 標準差 數值0次數 綜合 類別數據
所(D)說(VE) 殷海光 15 1 5 8 1 (1029.0, 0.0, 318.07

In [6]:
#計算各類別間的預測及OOB error
for file in os.listdir(input_path):
    with codecs.open(input_path+file,'rb','utf8') as f:
        content = f.readlines()
        
        print (file)
        
        temp_label = ''
        temp_index = ''
        if 'name' in file:
            temp_label = classification_name
            temp_index = name_index
        elif 'topic' in file:
            temp_label = classification_topic
            temp_index = topic_index
        
        accuracy = []
        vector_space = np.zeros((len(temp_index)+1,len(temp_index)+1),np.float64) #多平均
        
        for i in range(len(content)-1):
            if content[i][0] == '#':
                #accuracy.append((content[i][1:],float(content[i+1].split('：')[1]))) #accuracy
                accuracy.append((content[i][1:],float(content[i+2].split(':')[1]))) #oob error
                
        for i in accuracy:
            #print (i[0],round(i[1],3))
            vector_space[temp_index[i[0].split()[0]],temp_index[i[0].split()[1]]] = round(i[1],3)
            vector_space[temp_index[i[0].split()[1]],temp_index[i[0].split()[0]]] = round(i[1],3)
            
        for i in range(len(temp_index)):
            vector_space[i,-1] = round(sum(vector_space[i])/(len(temp_index)-1),3)
        for i in range(len(temp_index)):
            vector_space[-1,i] = round(sum(vector_space[:,i])/(len(temp_index)-1),3)
        vector_space[-1,-1] = round(sum(vector_space[-1])/(len(temp_index)),3)
        
        print (' '.join(temp_label+['平均']))
        print (vector_space)
        #df = DataFrame(vector_space,index=temp_label,columns=temp_label)
        print ()

FC bigram name.txt
雷震 殷海光 夏道平 傅正 龍平甫 蔣勻田 朱伴耘 胡適 羅鴻詔 平均
[[ 0.     0.068  0.226  0.154  0.     0.229  0.109  0.149  0.114  0.131]
 [ 0.068  0.     0.077  0.027  0.     0.043  0.023  0.044  0.     0.035]
 [ 0.226  0.077  0.     0.174  0.     0.036  0.075  0.074  0.     0.083]
 [ 0.154  0.027  0.174  0.     0.021  0.025  0.051  0.025  0.027  0.063]
 [ 0.     0.     0.     0.021  0.     0.105  0.054  0.036  0.037  0.032]
 [ 0.229  0.043  0.036  0.025  0.105  0.     0.125  0.062  0.065  0.086]
 [ 0.109  0.023  0.075  0.051  0.054  0.125  0.     0.043  0.068  0.068]
 [ 0.149  0.044  0.074  0.025  0.036  0.062  0.043  0.     0.022  0.057]
 [ 0.114  0.     0.     0.027  0.037  0.065  0.068  0.022  0.     0.042]
 [ 0.131  0.035  0.083  0.063  0.032  0.086  0.068  0.057  0.042  0.066]]

FC D+V name.txt
雷震 殷海光 夏道平 傅正 龍平甫 蔣勻田 朱伴耘 胡適 羅鴻詔 平均
[[ 0.     0.25   0.302  0.282  0.196  0.333  0.109  0.362  0.25   0.26 ]
 [ 0.25   0.     0.173  0.162  0.148  0.217  0.159  0.178  0.186  0.184]
 [ 0.302  0.173